<a href="https://colab.research.google.com/github/Ino54/MA_GreenAI-Practical-Experiments/blob/main/bloom_kd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Requirements
%%writefile requirements.txt
transformers>=4.41,<5
accelerate>=0.30
bitsandbytes>=0.43
datasets>=2.19
evaluate
sacrebleu
codecarbon>=2.5,<3
pynvml>=11.5.0
psutil
numpy
pandas
huggingface_hub
peft>=0.11
sentencepiece

Writing requirements.txt


In [ ]:
!pip -q install -r requirements.txt
!pip uninstall -y -q google-genai firebase-admin || true

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.6/517.6 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 118.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 9.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
firebase-admin 6.9.0 requires httpx[http2]==0.28.1, but you have httpx 0.27.2 which is incompatible.
google-genai 1.38.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incom

In [ ]:
# --- Google Drive mounten ---
import os, shutil, time, pathlib, inspect
from google.colab import drive
MOUNTPOINT="/content/drive"
already=os.path.isdir(os.path.join(MOUNTPOINT,"MyDrive"))
if not already and os.path.isdir(MOUNTPOINT) and os.listdir(MOUNTPOINT):
    backup=f"/content/drive_stale_{int(time.time())}"
    shutil.move(MOUNTPOINT, backup)
    os.makedirs(MOUNTPOINT, exist_ok=True)
drive.mount(MOUNTPOINT, force_remount=(not already))

# --- Zielordner (dein Pfad) ---
work_dir="/content/drive/MyDrive/LLM-Effizienz/4_3_Effizienzstrategien/bloom_kd"
if not os.path.isdir(work_dir):
    raise FileNotFoundError(
        f"Zielordner fehlt: {work_dir}\n"
        "Bitte Ordner in Google Drive anlegen und Zelle erneut ausführen."
    )
os.chdir(work_dir)
project_dir=work_dir
print("Arbeitsordner:", os.getcwd())

Mounted at /content/drive
Arbeitsordner: /content/drive/MyDrive/LLM-Effizienz/4_3_Effizienzstrategien/bloom_kd


In [ ]:
# --- Hugging Face Login ---
from google.colab import userdata
from huggingface_hub import login
hf_token = userdata.get("HF_TOKEN")
if hf_token:
    login(hf_token); print("Hugging Face Login erfolgreich!")
else:
    print("WARNUNG: Kein HF_TOKEN gefunden.")

Hugging Face Login erfolgreich!


In [ ]:
# ================== Imports & Setup ==================
import re, math, gc, platform, warnings
warnings.filterwarnings("ignore")
import numpy as np, pandas as pd, torch, psutil
from contextlib import nullcontext
from types import SimpleNamespace
from datasets import load_dataset
import evaluate
import torch.nn.functional as F
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    BitsAndBytesConfig, set_seed,
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
)
from codecarbon import EmissionsTracker
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

RESULT_BASENAME = "bloom_kd"
set_seed(42)
device="cuda" if torch.cuda.is_available() else "cpu"
if device=="cuda":
  gpu_name=torch.cuda.get_device_name(0)
  vram_total_gb=torch.cuda.get_device_properties(0).total_memory/(1024**3)
  torch.backends.cuda.matmul.allow_tf32=True
else:
  gpu_name="CPU"; vram_total_gb=0.0
print(f"Device: {device} | GPU: {gpu_name} | VRAM={vram_total_gb:.1f} GB | Torch {torch.__version__} | Py {platform.python_version()}")

Device: cuda | GPU: NVIDIA A100-SXM4-40GB | VRAM=39.6 GB | Torch 2.8.0+cu126 | Py 3.12.11


In [ ]:
# ================== CodeCarbon Helpers ==================
import pandas as _pd
def _cc_supported_kwargs():
    base=dict(log_level="error", output_dir=".", measure_power_secs=1, tracking_mode="process")
    try:
        params=inspect.signature(EmissionsTracker.__init__).parameters
        if "cloud_provider" in params: base["cloud_provider"]="google"
        if "cloud_region"  in params: base["cloud_region"]="europe-west10"
        if "country_iso_code" in params: base["country_iso_code"]="DEU"
    except Exception: pass
    return base
def make_tracker(name,out): return EmissionsTracker(project_name=name, output_file=out, **_cc_supported_kwargs())
def safe_start(tr):
    try: tr.start(); return True
    except Exception as e: print("[CodeCarbon] Start fehlgeschlagen:", e); return False
def safe_stop(tr, started):
    if not started: return SimpleNamespace(energy_consumed=0.0, emissions=0.0)
    try: return tr.stop()
    except Exception as e: print("[CodeCarbon] Stop fehlgeschlagen:", e); return SimpleNamespace(energy_consumed=0.0, emissions=0.0)
def unpack(em):
    if hasattr(em,"energy_consumed") and hasattr(em,"emissions"):
        try: return float(em.energy_consumed), float(em.emissions)
        except: pass
    if isinstance(em, dict):
        e=em.get("energy_consumed",0.0); c=em.get("emissions", em.get("emissions_kg",0.0))
        try: return float(e), float(c)
        except: return 0.0,0.0
    try: return 0.0, float(em)
    except: return 0.0,0.0
def read_energy_from_log(path):
    try:
        if not os.path.exists(path): return 0.0
        df=_pd.read_csv(path)
        for c in ["energy_consumed","energy_consumed_kwh","energy_consumed (kWh)","energy (kWh)"]:
            if c in df.columns: return float(df[c].iloc[-1])
        for c in df.columns:
            if "energy" in c.lower() and "kwh" in c.lower(): return float(df[c].iloc[-1])
    except: pass
    return 0.0
def measure_phase(phase, fn, prefix):
    logfile=os.path.join(project_dir, f"{prefix}_{phase}.csv")
    tr=make_tracker(f"{prefix}_{phase}", logfile)
    import time as _t
    st=safe_start(tr); t0=_t.time(); res=fn(); t1=_t.time()
    e=safe_stop(tr, st); ekwh, co2=unpack(e)
    if ekwh==0.0:
        ek=read_energy_from_log(logfile)
        if ek: ekwh=ek
    return {"phase":phase,"time_s":t1-t0,"energy_kwh":ekwh,"co2_kg":co2}, res

In [ ]:
# ================== KD/Eval Konfiguration ==================
TEACHER_ID="bigscience/bloom-3b"
STUDENT_ID="bigscience/bloom-560m"
KD_CFG = dict(
    teacher_8bit=True,
    student_4bit=True,
    use_lora=True,
    temperature=2.5,
    alpha=0.7,              # Endwert; Startwert unten
    alpha_start=0.9,        # Startwert für Schedule
    block_size=1024,        # ggf. auf 512 reduzieren falls VRAM knapp
    epochs=3,
    bsz=2,
    grad_accum=8,
    lr=1.5e-4,
    weight_decay=0.1,
    warmup_ratio=0.03,
    max_grad_norm=0.3,
    logging_steps=25,
    save_steps=500,
    train_split="train[:10%]",
    eval_split="test[:1%]",
)
EVAL={"max_new_tokens":32,
      "ppl":{"name":"wikitext","config":"wikitext-2-raw-v1","split":KD_CFG["eval_split"]},
      "bleu":{"name":"wmt14","config":"de-en","split":"test[:16]"}}
PROMPTS=[
  "Schreibe einen kurzen Absatz über nachhaltige KI.",
  "Erkläre in einfachen Worten, was Quantisierung in neuronalen Netzen ist.",
  "Nenne drei Vorteile von Mixture-of-Experts-Modellen."
]
bleu_metric=evaluate.load("sacrebleu")

In [ ]:
# ================== Helper (Gen/PPL/BLEU) ==================
def autocast_ctx():
  return torch.autocast(device_type="cuda", dtype=torch.float16) if device=="cuda" else nullcontext()
def safe_max_len(tok, model, fallback=2048, upper=100000):
  cand=getattr(tok,"model_max_length",None)
  if isinstance(cand,int) and 0<cand<upper: return cand
  cand=getattr(getattr(model,"config",None),"max_position_embeddings",None)
  if isinstance(cand,int) and 0<cand<upper: return cand
  return fallback
def warmup(model, tok, max_len):
  with torch.no_grad(), autocast_ctx():
    x=tok("Hello", return_tensors="pt", truncation=True, max_length=max_len).to(model.device)
    _=model.generate(**x, max_new_tokens=1, do_sample=False, pad_token_id=tok.eos_token_id)

def do_gen(model, tok, max_new_tokens):
  total, texts=0, []
  ml=safe_max_len(tok, model)
  for p in PROMPTS:
    enc=tok(p, return_tensors="pt", truncation=True, max_length=ml).to(model.device)
    room=ml-enc["input_ids"].shape[1]
    cur_new=max(1, min(max_new_tokens, int(room)))
    with torch.no_grad(), autocast_ctx():
      out=model.generate(**enc, max_new_tokens=cur_new, do_sample=False, pad_token_id=tok.eos_token_id)
    total+=int(out.shape[1]-enc["input_ids"].shape[1])
    texts.append(tok.decode(out[0], skip_special_tokens=True))
  return texts, total

def do_ppl(model, tok, ds_cfg):
  ds=load_dataset(ds_cfg["name"], ds_cfg["config"], split=ds_cfg["split"])
  ml=safe_max_len(tok, model); losses=[]
  with torch.no_grad():
    for t in ds["text"]:
      if not isinstance(t,str) or len(t.strip())<4: continue
      enc=tok(t, return_tensors="pt", truncation=True, max_length=ml).to(model.device)
      with autocast_ctx():
        out=model(enc["input_ids"], labels=enc["input_ids"])
      losses.append(float(out.loss.detach().cpu()))
  return math.exp(np.mean(losses)) if losses else None

def do_bleu(model, tok, ds_cfg, max_new_tokens):
  ds=load_dataset(ds_cfg["name"], ds_cfg["config"], split=ds_cfg["split"])
  ml=safe_max_len(tok, model); preds, refs=[], []
  with torch.no_grad():
    for ex in ds:
      de,en=ex["translation"]["de"], ex["translation"]["en"]
      prompt=f"Translate to English:\nGerman: {de}\nEnglish:"
      enc=tok(prompt, return_tensors="pt", truncation=True, max_length=ml).to(model.device)
      room=ml-enc["input_ids"].shape[1]
      cur_new=max(1, min(max_new_tokens, int(room)))
      with autocast_ctx():
        out=model.generate(**enc, max_new_tokens=cur_new, do_sample=False, pad_token_id=tok.eos_token_id)
      gen=tok.decode(out[0], skip_special_tokens=True)
      hyp=gen.split("English:")[-1].strip().split("\n")[0].strip() or gen.strip()
      preds.append(hyp); refs.append([en])
  return float(bleu_metric.compute(predictions=preds, references=refs)["score"])

In [ ]:
 #================== KD/LoRA ==================
def maybe_wrap_lora(model, r=32, alpha=64, dropout=0.05):
    target_modules = ["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h"]  # BLOOM
    cfg = LoraConfig(
        r=r, lora_alpha=alpha, lora_dropout=dropout,
        target_modules=target_modules, bias="none",
        task_type=TaskType.CAUSAL_LM,
    )
    model = get_peft_model(model, cfg)
    try: model.print_trainable_parameters()
    except: pass
    return model

def kd_loss(student_logits, teacher_logits, labels, T: float, alpha: float):
    sT = student_logits[..., :-1, :] / T
    tT = teacher_logits[..., :-1, :] / T
    lab = labels[..., 1:]
    kd = F.kl_div(F.log_softmax(sT, dim=-1), F.softmax(tT, dim=-1), reduction="batchmean") * (T*T)
    ce = F.cross_entropy(student_logits[..., :-1, :].transpose(1, 2), lab, ignore_index=-100)
    return alpha*kd + (1.0-alpha)*ce

class KDTrainer(Trainer):
    def __init__(self, *args, teacher=None, temperature=2.5, alpha=0.7, alpha_start=0.9, **kwargs):
        super().__init__(*args, **kwargs)
        assert teacher is not None, "Teacher-Modell erforderlich."
        self.teacher = teacher.eval()
        for p in self.teacher.parameters(): p.requires_grad_(False)
        self.temperature = temperature
        self.alpha = alpha
        self.alpha_start = alpha_start

    def _alpha_now(self):
        ep = getattr(self.state, "epoch", None)
        if ep is None or self.args.num_train_epochs <= 1:
            return self.alpha
        ratio = min(max(ep / self.args.num_train_epochs, 0.0), 1.0)
        return (1 - ratio) * self.alpha_start + ratio * self.alpha

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None, **kwargs):
        labels = inputs.get("labels", None)
        if labels is None:
            labels = inputs["input_ids"].clone()
            if "attention_mask" in inputs:
                labels[inputs["attention_mask"]==0] = -100

        with torch.no_grad():
            t_out = self.teacher(
                input_ids=inputs["input_ids"],
                attention_mask=inputs.get("attention_mask")
            )
        s_out = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs.get("attention_mask")
        )
        s_logits, t_logits = s_out.logits, t_out.logits
        T = self.temperature
        alpha_now = self._alpha_now()

        sT = s_logits[..., :-1, :] / T
        tT = t_logits[..., :-1, :] / T
        lab = labels[..., 1:]

        kd = F.kl_div(F.log_softmax(sT, dim=-1), F.softmax(tT, dim=-1), reduction="batchmean") * (T*T)
        ce = F.cross_entropy(s_logits[..., :-1, :].transpose(1, 2), lab, ignore_index=-100)
        loss = alpha_now*kd + (1.0-alpha_now)*ce
        return (loss, s_out) if return_outputs else loss

In [ ]:
# ================== Datenaufbereitung ==================
def build_datasets(tokenizer, dataset="wikitext", config="wikitext-2-raw-v1",
                   train_split=KD_CFG["train_split"], eval_split=KD_CFG["eval_split"],
                   block_size=KD_CFG["block_size"]):
    ds_train = load_dataset(dataset, config, split=train_split)
    ds_eval  = load_dataset(dataset, config, split=eval_split)
    def tok_fn(batch):
        return tokenizer(batch["text"], return_attention_mask=False, truncation=False)
    train_tok = ds_train.map(tok_fn, batched=True, remove_columns=ds_train.column_names)
    eval_tok  = ds_eval.map(tok_fn,  batched=True, remove_columns=ds_eval.column_names)
    def group_texts(examples):
        concatenated = sum(examples["input_ids"], [])
        total_len = (len(concatenated) // block_size) * block_size
        concatenated = concatenated[:total_len]
        return {"input_ids": [concatenated[i:i+block_size] for i in range(0, total_len, block_size)]}
    train_blocks = train_tok.map(group_texts, batched=True)
    eval_blocks  = eval_tok.map(group_texts,  batched=True)
    return train_blocks, eval_blocks

In [ ]:
 # ================== Modelle laden ==================
def load_teacher_student():
    tok = AutoTokenizer.from_pretrained(STUDENT_ID, use_fast=True)
    if tok.pad_token is None: tok.pad_token = tok.eos_token
    tok.padding_side="right"

    t_bnb = BitsAndBytesConfig(load_in_8bit=True) if KD_CFG["teacher_8bit"] else None
    teacher = AutoModelForCausalLM.from_pretrained(
        TEACHER_ID, device_map="auto",
        quantization_config=t_bnb
    ).eval()
    for p in teacher.parameters():
        p.requires_grad_(False)

    s_bnb = None
    if KD_CFG["student_4bit"]:
        s_bnb = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16
        )
    student = AutoModelForCausalLM.from_pretrained(
        STUDENT_ID,
        device_map="auto" if (KD_CFG["student_4bit"] or KD_CFG["teacher_8bit"]) else None,
        quantization_config=s_bnb
    )

    student = prepare_model_for_kbit_training(student, use_gradient_checkpointing=False)
    student.config.use_cache = False

    if KD_CFG["use_lora"]:
        student = maybe_wrap_lora(student, r=32, alpha=64, dropout=0.05)

    try:
        student.enable_input_require_grads()
    except Exception:
        pass

    student.train()

    trainable = sum(p.numel() for p in student.parameters() if p.requires_grad)
    if trainable == 0:
        raise RuntimeError("Keine trainierbaren Parameter im Student! Prüfe LoRA-Targets/QLoRA-Setup.")
    print(f"[Sanity] Trainable params: {trainable:,}")

    return tok, teacher, student

In [ ]:
# ================== TrainingArguments (robust gegen API-Änderungen) ==================
def build_training_args(output_dir, bf16_ok):
    # gewünschte Defaults
    desired = dict(
        output_dir=output_dir,
        per_device_train_batch_size=KD_CFG["bsz"],
        gradient_accumulation_steps=KD_CFG["grad_accum"],
        learning_rate=KD_CFG["lr"],
        num_train_epochs=KD_CFG["epochs"],
        logging_steps=KD_CFG["logging_steps"],
        save_steps=KD_CFG["save_steps"],
        save_total_limit=2,
        report_to="none",
        fp16=not bf16_ok, bf16=bf16_ok,
        gradient_checkpointing=False,
        ddp_find_unused_parameters=False,
        # folgende sind versionsabhängig:
        evaluation_strategy="epoch",   # alias: eval_strategy (neuere Versionen)
        optim="adamw_bnb_8bit",
        lr_scheduler_type="cosine",
        warmup_ratio=KD_CFG["warmup_ratio"],
        weight_decay=KD_CFG["weight_decay"],
        max_grad_norm=KD_CFG["max_grad_norm"],
        logging_first_step=True
    )
    params = inspect.signature(TrainingArguments.__init__).parameters
    # Map/Drop je nach Signatur
    if "evaluation_strategy" not in params:
        if "eval_strategy" in params:
            desired["eval_strategy"] = desired.pop("evaluation_strategy")
        else:
            desired.pop("evaluation_strategy", None)
    if "optim" not in params:
        desired.pop("optim", None)
    if "logging_first_step" not in params:
        desired.pop("logging_first_step", None)
    # Nur erlaubte Keys übergeben (failsafe)
    filtered = {k:v for k,v in desired.items() if k in params}
    return TrainingArguments(**filtered)

# ================== Training + Eval orchestrieren ==================
def train_kd_once():
    tok, teacher, student = load_teacher_student()
    train_ds, eval_ds = build_datasets(tok)
    collator = DataCollatorForLanguageModeling(tok, mlm=False)

    bf16_ok = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8
    args = build_training_args(os.path.join(project_dir, "train_out"), bf16_ok)

    trainer = KDTrainer(
        model=student, teacher=teacher,
        temperature=KD_CFG["temperature"], alpha=KD_CFG["alpha"], alpha_start=KD_CFG["alpha_start"],
        args=args, train_dataset=train_ds, eval_dataset=eval_ds, data_collator=collator
    )
    trainer.train()

    # Speichern (bei QLoRA/LoRA: Adapter)
    save_dir=os.path.join(project_dir, "student_kd")
    os.makedirs(save_dir, exist_ok=True)
    student.save_pretrained(save_dir)
    tok.save_pretrained(save_dir)

    # Eval & Generation (kurz)
    ml=safe_max_len(tok, student); warmup(student, tok, ml)
    ppl = do_ppl(student, tok, EVAL["ppl"])
    bleu = do_bleu(student, tok, EVAL["bleu"], EVAL["max_new_tokens"])
    samples, n_tok = do_gen(student, tok, EVAL["max_new_tokens"])

    with open(os.path.join(project_dir, f"{RESULT_BASENAME}_samples_student.txt"),"w",encoding="utf-8") as f:
        for i,t in enumerate(samples,1): f.write(f"--- Beispiel {i} ---\n{t}\n\n")

    return {"ppl": ppl, "bleu": bleu, "tokens_out": int(n_tok), "save_dir": save_dir}, samples, tok, student

In [ ]:
# ================== Ausführung (mit CodeCarbon pro Phase) ==================
prefix=f"{RESULT_BASENAME}_b560_from_b3b_qlo"
m_train, (train_res, samples, tok, student) = measure_phase("train", train_kd_once, prefix)
m_ppl, _  = measure_phase("eval_ppl",  lambda: do_ppl(student, tok, EVAL["ppl"]), prefix)
m_bleu, _ = measure_phase("eval_bleu", lambda: do_bleu(student, tok, EVAL["bleu"], EVAL["max_new_tokens"]), prefix)
m_gen, _  = measure_phase("gen",       lambda: do_gen(student, tok, EVAL["max_new_tokens"]), prefix)

# ================== Ergebnisse sammeln & speichern ==================
row = dict(
    teacher_id=TEACHER_ID, student_id=STUDENT_ID,
    kd_temperature=KD_CFG["temperature"], kd_alpha=KD_CFG["alpha"],
    block_size=KD_CFG["block_size"], epochs=KD_CFG["epochs"],
    ppl=train_res["ppl"], bleu=train_res["bleu"], tokens_out=train_res["tokens_out"],
    notes=f"GPU={gpu_name}, VRAM={vram_total_gb:.1f} GB; Teacher 8-bit={KD_CFG['teacher_8bit']}; Student 4-bit={KD_CFG['student_4bit']}; LoRA={KD_CFG['use_lora']}"
)
totals = dict(
    time_s = m_train["time_s"] + m_ppl["time_s"] + m_bleu["time_s"] + m_gen["time_s"],
    energy_kwh = m_train["energy_kwh"] + m_ppl["energy_kwh"] + m_bleu["energy_kwh"] + m_gen["energy_kwh"],
    co2_kg = m_train["co2_kg"] + m_ppl["co2_kg"] + m_bleu["co2_kg"] + m_gen["co2_kg"]
)
row.update({
    "time_s_total": totals["time_s"],
    "energy_kwh_total": totals["energy_kwh"],
    "co2_kg_total": totals["co2_kg"],
    "kg_per_kwh": (totals["co2_kg"]/totals["energy_kwh"]) if totals["energy_kwh"] else None
})

df = pd.DataFrame([row])
df_phases = pd.DataFrame([m_train, m_ppl, m_bleu, m_gen])
df.to_csv(os.path.join(project_dir, f"{RESULT_BASENAME}_results.csv"), index=False)
df_phases.to_csv(os.path.join(project_dir, f"{RESULT_BASENAME}_results_per_phase.csv"), index=False)

print("\n=== KD Ergebnisse (Student) ===\n", df)
print("\n=== Phasen (Zeit/Energie/CO2) ===\n", df_phases)
print("Gespeichert:", os.path.join(project_dir, f"{RESULT_BASENAME}_results.csv"))

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.01G [00:00<?, ?B/s]

...

de-en/test-00000-of-00001.parquet:   0%|          | 0.00/509k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4508785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]


=== KD Ergebnisse (Student) ===
             teacher_id             student_id  kd_temperature  kd_alpha  \
0  bigscience/bloom-3b  bigscience/bloom-560m             2.5       0.7   

   block_size  epochs         ppl      bleu  tokens_out  \
0        1024       3  329.866691  1.197423          96   

                                               notes  time_s_total  \
0  GPU=NVIDIA A100-SXM4-40GB, VRAM=39.6 GB; Teach...    588.259584   

   energy_kwh_total  co2_kg_total  kg_per_kwh  
0          0.026248      0.012357    0.470783  

=== Phasen (Zeit/Energie/CO2) ===
        phase      time_s  energy_kwh    co2_kg
0      train  541.466463    0.024993  0.011766
1   eval_ppl    4.675794    0.000124  0.000058
2  eval_bleu   35.884409    0.000963  0.000454
3        gen    6.232917    0.000168  0.000079
Gespeichert: /content/drive/MyDrive/LLM-Effizienz/4_3_Effizienzstrategien/bloom_kd/bloom_kd_results.csv
